<a href="https://colab.research.google.com/github/AlirezaFazli29/AC-GAN/blob/main/Whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Whisper

## Libraries

In [1]:
! pip install --upgrade pip
# ! pip install --upgrade git+https:/|/github.com/huggingface/transformers.git accelerate datasets[audio]
! pip install torchinfo
! pip install pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [55]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import matplotlib.pyplot as plt
# from IPython.display import Audio
from torchinfo import summary
# import soundfile as sf
import numpy as np
from pydub import AudioSegment
import os
import zipfile
import gzip
import glob
from shutil import make_archive

## Default Device

In [3]:
if torch.cuda.is_available(): print(torch.cuda.get_device_name())
else: print('cpu')
default_device = 'cuda' if torch.cuda.is_available() else 'cpu'

Tesla T4


## Model

In [4]:
model_id = "openai/whisper-large-v3"
batch_size = 16
model = AutoModelForSpeechSeq2Seq.from_pretrained(pretrained_model_name_or_path=model_id).to(default_device)
processor = AutoProcessor.from_pretrained(model_id)
pipe = pipeline("automatic-speech-recognition",
                model=model,
                tokenizer=processor.tokenizer,
                feature_extractor=processor.feature_extractor,
                max_new_tokens=128,
                chunk_length_s=30,
                batch_size=batch_size,
                return_timestamps=True,
                device=default_device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
summary(model)

Layer (type:depth-idx)                                  Param #
WhisperForConditionalGeneration                         --
├─WhisperModel: 1-1                                     --
│    └─WhisperEncoder: 2-1                              --
│    │    └─Conv1d: 3-1                                 492,800
│    │    └─Conv1d: 3-2                                 4,916,480
│    │    └─Embedding: 3-3                              (1,920,000)
│    │    └─ModuleList: 3-4                             629,637,120
│    │    └─LayerNorm: 3-5                              2,560
│    └─WhisperDecoder: 2-2                              --
│    │    └─Embedding: 3-6                              66,388,480
│    │    └─WhisperPositionalEmbedding: 3-7             573,440
│    │    └─ModuleList: 3-8                             839,557,120
│    │    └─LayerNorm: 3-9                              2,560
├─Linear: 1-2                                           66,388,480
Total params: 1,609,879,040
Trainable params

## Dataset

In [6]:
zip_file_path = './Recordings.zip'
extracted_folder_path = './Recordings'

In [7]:
if (os.path.exists(extracted_folder_path) and os.path.isdir(extracted_folder_path)):
    print(f'folder {extracted_folder_path} exists')
else:
    with zipfile.ZipFile(zip_file_path, mode='r') as zip_ref:
        zip_ref.extractall(extracted_folder_path)
    print('files extracted')

files extracted


In [8]:
paths = glob.glob(extracted_folder_path+'/*')
len(paths)

106

## Using the model

In [16]:
results = [None for _ in range(len(paths))]

for i, path in enumerate(paths):

    audio = AudioSegment.from_file(path)
    waveform = np.array(audio.normalize().set_channels(1).get_array_of_samples())
    waveform = waveform/max(waveform)
    sr = audio.frame_rate
    results[i] = pipe({'array': waveform, 'sampling_rate':sr})

Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the m

## Save Transcriptions

In [22]:
transcriptions_folder = './Transcriptions'

In [23]:
if (os.path.exists(transcriptions_folder) and os.path.isdir(transcriptions_folder)):
    print(f'folder {transcriptions_folder} exists')
else:
    os.mkdir(transcriptions_folder)
    print('Folder created')

Folder created


In [41]:
transcription_paths = [transcriptions_folder+'/'+path.split('/')[-1].split('.')[0]+'.txt' for path in paths]

In [43]:
for i, path in enumerate(transcription_paths):
    text_file = open(path, "w")
    text_file.write(results[i]['text'])
    text_file.close()

In [58]:
make_archive(transcriptions_folder, 'zip', transcriptions_folder)

'/content/Transcriptions.zip'